<a href="https://colab.research.google.com/github/aydinmyilmaz/Spark-NLP/blob/master/GerMed__Data_Preprocessing_in_SPARK_NLP_and_Word_Embeddings_3_1509.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4
! pip install --ignore-installed -q spark-nlp==2.6.1

openjdk version "1.8.0_265"
OpenJDK Runtime Environment (build 1.8.0_265-8u265-b01-0ubuntu2~18.04-b01)
OpenJDK 64-Bit Server VM (build 25.265-b01, mixed mode)
     |████████████████████████████████| 215.7MB 61kB/s 
     |████████████████████████████████| 204kB 19.2MB/s 
     |████████████████████████████████| 133kB 2.8MB/s 


In [2]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())

print("Apache Spark version:", spark.version)

Spark NLP version 2.6.1
Apache Spark version: 2.4.4


# 1. SPARK Pipeline for Preprocessing

In [5]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetector().\
    setInputCols(['document']).\
    setOutputCol('sentences')

tokenizer = Tokenizer() \
    .setInputCols(['sentences']) \
    .setOutputCol("token")

pattern='^.*(?=.{8,})(?=.*[a-zA-ZäöüÄÖÜß])(?=.*\d).*$'
pattern2 = '''[~!@#$^%&*\\(\\)_+={}\\[\\]|;:\"'<,>.?`/\\\\–[0-9]]'''
#pattern2 = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("norma")\
    .setLowercase(False)\
    .setCleanupPatterns([pattern,pattern2])

stopwords_cleaner = StopWordsCleaner.pretrained(name='stopwords_de', lang='de')\
      .setInputCols("norma")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

lemmatizer = LemmatizerModel.pretrained(name='lemma', lang='de') \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemma") \

tokenassembler = TokenAssembler()\
    .setInputCols(["sentences", "lemma"]) \
    .setOutputCol("clean_text")


pipeline = Pipeline(
    stages = [documentAssembler,
              sentenceDetector,
              tokenizer,
              normalizer,
              stopwords_cleaner,
              lemmatizer,
              tokenassembler
            ])


empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = pipeline.fit(empty_df)

stopwords_de download started this may take some time.
Approximate size to download 2.9 KB
[OK!]
lemma download started this may take some time.
Approximate size to download 4 MB
[OK!]


### 1.2.Short Pipeline

In [88]:
# without lemma and stopwords cleaning

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetector().\
    setInputCols(['document']).\
    setOutputCol('sentences')

tokenizer = Tokenizer() \
    .setInputCols(['sentences']) \
    .setOutputCol("token")

pattern='^.*(?=.{8,})(?=.*[a-zA-ZäöüÄÖÜß])(?=.*\d).*$'
pattern2 = '''[~!@#$^%&*\\(\\)_+={}\\[\\]|;:\"'<,>.?`/\\\\–[0-9]]'''
#pattern2 = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("norma")\
    .setLowercase(False)\
    .setCleanupPatterns([pattern,pattern2])

tokenassembler = TokenAssembler()\
    .setInputCols(["sentences", "norma"]) \
    .setOutputCol("clean_text")


pipeline = Pipeline(
    stages = [documentAssembler,
              sentenceDetector,
              tokenizer,
              normalizer,
              tokenassembler
            ])


empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel_2 = pipeline.fit(empty_df)

# 2. Pdf-->Text Data Preprocessing

In [113]:
from google.colab import files
uploaded = files.upload()

Saving ordered_txt_12.txt to ordered_txt_12.txt


In [114]:
spark_df = spark.read.text('./ordered_txt_12.txt').toDF('text')

spark_df.show(10)

+--------------------+
|                text|
+--------------------+
|Eine 45-jährige R...|
|.Ein 20-jähriger ...|
+--------------------+



2.1. Transform spark_df to pipeline

In [115]:
result = pipelineModel_2.transform(spark_df)
result.show(truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|                                                                                                text|                                                                                            document|                                                                                           sentences|                                                                                        

In [117]:
result.printSchema()

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentences: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = tru

In [118]:

import pyspark.sql.functions as F

pandas_df = result.withColumn(
    "tmp", 
    F.explode("clean_text")) \
    .select("tmp.*").select("begin","end","result","metadata.sentence").toPandas()
pandas_df.head()

,begin,end,result,sentence
0,0,222,Eine Richterin wird notärztlich eingewiesen da...,0
1,238,334,Noch im Flur muss sich die Patientin bei anhal...,1
2,337,382,Wesentliche Vorerkrankungen sind nicht bekannt,2
3,385,438,Ein regelmäßiger Alkoholkonsum wird glaubhaft ...,3
4,442,555,In der Jugend sei eine Appendektomie Frau in s...,4


In [119]:
pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33640 entries, 0 to 33639
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   begin     33640 non-null  int32 
 1   end       33640 non-null  int32 
 2   result    33640 non-null  object
 3   sentence  33640 non-null  object
dtypes: int32(2), object(2)
memory usage: 788.6+ KB


In [120]:
sent_df = pandas_df

In [121]:
sent_df['lang'] = sent_df.apply(lambda x : (int(x['end']))-(int(x['begin'])),axis=1) 
sent_df['word_num'] = sent_df.result.apply(lambda x : len(x.split()))
sent_df['word_list'] = sent_df.result.apply(lambda x : x.split())
sent_df = sent_df[sent_df.word_num>3]

sent_df.head() 

,begin,end,result,sentence,lang,word_num,word_list
0,0,222,Eine Richterin wird notärztlich eingewiesen da...,0,222,31,"[Eine, Richterin, wird, notärztlich, eingewies..."
1,238,334,Noch im Flur muss sich die Patientin bei anhal...,1,96,16,"[Noch, im, Flur, muss, sich, die, Patientin, b..."
2,337,382,Wesentliche Vorerkrankungen sind nicht bekannt,2,45,5,"[Wesentliche, Vorerkrankungen, sind, nicht, be..."
3,385,438,Ein regelmäßiger Alkoholkonsum wird glaubhaft ...,3,53,6,"[Ein, regelmäßiger, Alkoholkonsum, wird, glaub..."
4,442,555,In der Jugend sei eine Appendektomie Frau in s...,4,113,18,"[In, der, Jugend, sei, eine, Appendektomie, Fr..."


In [122]:
sent_corpus_1 = sent_df.word_list.tolist()
sent_corpus_1[0:5]

[['Eine',
  'Richterin',
  'wird',
  'notärztlich',
  'eingewiesen',
  'da',
  'sie',
  'nach',
  'dem',
  'Mittagessen',
  'plötzlich',
  'starke',
  'Bauchschmerzen',
  'verspürt',
  'habe',
  'die',
  'zunächst',
  'gürtelförmig',
  'bis',
  'in',
  'den',
  'Rücken',
  'und',
  'dann',
  'diffus',
  'in',
  'den',
  'gesamten',
  'Oberbauch',
  'ausgestrahlt',
  'hätten'],
 ['Noch',
  'im',
  'Flur',
  'muss',
  'sich',
  'die',
  'Patientin',
  'bei',
  'anhaltender',
  'Übelkeit',
  'nun',
  'schon',
  'zum',
  'dritten',
  'Mal',
  'erbrechen'],
 ['Wesentliche', 'Vorerkrankungen', 'sind', 'nicht', 'bekannt'],
 ['Ein', 'regelmäßiger', 'Alkoholkonsum', 'wird', 'glaubhaft', 'verneint'],
 ['In',
  'der',
  'Jugend',
  'sei',
  'eine',
  'Appendektomie',
  'Frau',
  'in',
  'schmerzbedingt',
  'reduziertem',
  'AZ',
  'und',
  'leicht',
  'übergewichtigem',
  'EZ',
  'cm',
  'kg',
  'BMI']]

In [123]:
len(sent_corpus_1)

28116

#  3.Med1_Data_Preprocessing

In [6]:
from google.colab import files
uploaded = files.upload()

Saving all_ger_med_text.txt to all_ger_med_text.txt


In [9]:
spark_df_2 = spark.read.text('./all_ger_med_text.txt').toDF('text')

spark_df_2.show(10)

+--------------------+
|                text|
+--------------------+
|Wäre zu schön, we...|
|Unabhängig davon ...|
|uuund.. ist es WI...|
|Löwenzahnsalat so...|
|Bewundere Dich no...|
|Ganz unabhängig d...|
|Wollte mich dann ...|
|Diese Eiweißaussc...|
|Ich werde jetzt d...|
|Was ich bemerkt h...|
+--------------------+
only showing top 10 rows



In [89]:
result_2 = pipelineModel_2.transform(spark_df_2)
result_2.show(truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|                                                                                                text|                                                                                            document|                                                                                           sentences|                                                                                        

In [91]:
import pyspark.sql.functions as F

pandasDf_2= result_2.withColumn(
    "tmp", 
    F.explode("clean_text")) \
    .select("tmp.*").select("begin","end","result","metadata.sentence").toPandas()
pandasDf_2.head()

,begin,end,result,sentence
0,0,214,Wäre zu schön wenn es da keine Probleme gegebe...,0
1,0,65,Unabhängig davon würde ich das aber mit den Ni...,0
2,68,224,Erst recht wenn die Essstörung seit der Behand...,1
3,0,4,uuund,0
4,6,5,,1


In [92]:
sent_df_2 = pandasDf_2

In [93]:
sent_df_2['lang'] = sent_df_2.apply(lambda x : (int(x['end']))-(int(x['begin'])),axis=1) 
sent_df_2['word_num'] = sent_df_2.result.apply(lambda x : len(x.split()))
sent_df_2['word_list'] = sent_df_2.result.apply(lambda x : x.split())
sent_df_2 = sent_df_2[sent_df_2.word_num>3]

sent_df_2.head() 

,begin,end,result,sentence,lang,word_num,word_list
0,0,214,Wäre zu schön wenn es da keine Probleme gegebe...,0,214,34,"[Wäre, zu, schön, wenn, es, da, keine, Problem..."
1,0,65,Unabhängig davon würde ich das aber mit den Ni...,0,65,11,"[Unabhängig, davon, würde, ich, das, aber, mit..."
2,68,224,Erst recht wenn die Essstörung seit der Behand...,1,156,22,"[Erst, recht, wenn, die, Essstörung, seit, der..."
5,8,93,ist es WIRLKICH Puls oder war das nur so eine ...,2,85,15,"[ist, es, WIRLKICH, Puls, oder, war, das, nur,..."
6,102,198,Wenns doch eher nur - war dann rückt Psyche sc...,3,96,17,"[Wenns, doch, eher, nur, -, war, dann, rückt, ..."


In [94]:
sent_df_2.sample(10)

,begin,end,result,sentence,lang,word_num,word_list
129431,534,594,Leider verfällt dieses tolle ereignis nach ein...,6,60,8,"[Leider, verfällt, dieses, tolle, ereignis, na..."
76223,177,276,Wenn auch ein zweiter oder dritter Wert gut au...,2,99,16,"[Wenn, auch, ein, zweiter, oder, dritter, Wert..."
154694,481,578,Oft war es auch der Wochenanfang weil ich fast...,5,97,17,"[Oft, war, es, auch, der, Wochenanfang, weil, ..."
231236,0,159,Nun frage ich mich aber ob so eine Steissbeinf...,0,159,28,"[Nun, frage, ich, mich, aber, ob, so, eine, St..."
84888,74,107,das merkt man dann schon denke ich,3,33,7,"[das, merkt, man, dann, schon, denke, ich]"
138526,0,84,Meine Ärztin meinte ich könnte eine Woche wart...,0,84,13,"[Meine, Ärztin, meinte, ich, könnte, eine, Woc..."
61001,54,217,Heute war ich zB in der Berufsschulpause im Mc...,1,163,27,"[Heute, war, ich, zB, in, der, Berufsschulpaus..."
262547,803,883,Vielleicht sprecht ihr mal mit euren Ärzten we...,11,80,13,"[Vielleicht, sprecht, ihr, mal, mit, euren, Är..."
27239,114,130,Es gibt dann evtl,2,16,4,"[Es, gibt, dann, evtl]"
127427,0,138,Die Bandage nennt sich Handgelenkriemen mit Da...,0,138,18,"[Die, Bandage, nennt, sich, Handgelenkriemen, ..."


In [95]:
sent_corpus_2 = sent_df_2.word_list.tolist()
#print(sent_corpus[100])
len(sent_corpus_2)

235925

In [96]:
sent_corpus_2[0:10]

[['Wäre',
  'zu',
  'schön',
  'wenn',
  'es',
  'da',
  'keine',
  'Probleme',
  'gegeben',
  'hätte',
  'die',
  'Vollidioten',
  'haben',
  'dabei',
  'die',
  'Blutversorgung',
  'der',
  'neuen',
  'Niere',
  'verletzt',
  'und',
  'die',
  'Niere',
  'war',
  'fast',
  'Stunden',
  'unversorgt',
  'weil',
  'man',
  'auf',
  'einen',
  'Gefäßchirurgen',
  'warten',
  'musste'],
 ['Unabhängig',
  'davon',
  'würde',
  'ich',
  'das',
  'aber',
  'mit',
  'den',
  'Nieren',
  'abklären',
  'lassen'],
 ['Erst',
  'recht',
  'wenn',
  'die',
  'Essstörung',
  'seit',
  'der',
  'Behandlung',
  'nicht',
  'komplett',
  'weg',
  'ist',
  'und',
  'sich',
  'weiterhin',
  'in',
  'deiner',
  'Ernährung',
  'und',
  'Flüssigkeitszufuhr',
  'niederschlagen',
  'sollte'],
 ['ist',
  'es',
  'WIRLKICH',
  'Puls',
  'oder',
  'war',
  'das',
  'nur',
  'so',
  'eine',
  'Umschreibung',
  'für',
  'wirklich',
  'schnellen',
  'Puls'],
 ['Wenns',
  'doch',
  'eher',
  'nur',
  '-',
  'war',
  

In [124]:
sent_corpus_3 = sent_corpus_1 + sent_corpus_2
len(sent_corpus_3)

264041

In [125]:
len(sent_corpus_2)

235925

In [127]:
sent_corpus_pd = pd.DataFrame(sent_corpus_3)
sent_corpus_pd.to_csv("sent_corpus_3.csv")

# 4. Word Embeddings Model 

In [132]:
# Loading libraries
import time
import pandas as pd
from gensim.models import Word2Vec

def cal_elapsed_time(s):
    return print("Elapsed time:\t", round((time.time() - s),2))
s_time = time.time()
cal_elapsed_time(s=s_time)

s_time = time.time()
print("Model Training Started...")
model_w2v = Word2Vec(sentences=sent_corpus_2, size=300, window=4, min_count=1, workers=4) #sg=1
print('time :', cal_elapsed_time(s_time))

print("Total number of unique words loaded in Model : ", len(model_w2v.wv.vocab))



Elapsed time:	 0.0
Model Training Started...
Elapsed time:	 70.73
time : None
Total number of unique words loaded in Model :  127398


In [131]:
model_w2v.save("word2vec_germed_model300_v6_mergedcorpus.bin")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [129]:
model_w2v.wv.most_similar("Woche", topn=10)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('woche', 0.8980359435081482),
 ('Stunde', 0.7820115089416504),
 ('Nacht', 0.7367208003997803),
 ('wochen', 0.7240251302719116),
 ('Wochen', 0.7126184701919556),
 ('tage', 0.707195520401001),
 ('Tage', 0.6922171115875244),
 ('zeit', 0.6884756088256836),
 ('op', 0.6756557822227478),
 ('Jahr', 0.658408522605896)]

In [ ]:
[('woche', 0.8980359435081482),
 ('Stunde', 0.7820115089416504),
 ('Nacht', 0.7367208003997803),
 ('wochen', 0.7240251302719116),
 ('Wochen', 0.7126184701919556),
 ('tage', 0.707195520401001),
 ('Tage', 0.6922171115875244),
 ('zeit', 0.6884756088256836),
 ('op', 0.6756557822227478),
 ('Jahr', 0.658408522605896)]
 
[('woche', 0.9124968647956848),
 ('Stunde', 0.7964605093002319),
 ('Nacht', 0.7602499723434448),
 ('Wochen', 0.7167088985443115),
 ('wochen', 0.7163565158843994),
 ('zeit', 0.710381031036377),
 ('Weile', 0.6988216638565063),
 ('tage', 0.6961817741394043),
 ('Tage', 0.6799865961074829),
 ('Monat', 0.6722192764282227)]

# Fasttext Word Embeddings

In [135]:
from gensim.models import FastText
model_fst = FastText(sentences=sent_corpus_3, size=300, window=4, min_count=1, workers=4)

print("Total number of unique words loaded in Model : ", len(model_fst.wv.vocab))



Total number of unique words loaded in Model :  142188


In [136]:
model_fst.wv.most_similar("Woche", topn=10)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('×Woche', 0.9753472805023193),
 ('SS-Woche', 0.975199282169342),
 ('mgWoche', 0.9744548797607422),
 ('OleovitWoche', 0.9721533060073853),
 ('MalWoche', 0.9629256725311279),
 ('TageWoche', 0.9615007638931274),
 ('woche', 0.9582335948944092),
 ('StundenWoche', 0.9453002214431763),
 ('Stunden-Woche', 0.943954586982727),
 ('WocheAngela', 0.9419333338737488)]

In [137]:
model_fst.save("fst_germed_model300_v6_mergedcorpus.bin")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
[('mgWoche', 0.9802114367485046),
 ('SS-Woche', 0.9787499904632568),
 ('OleovitWoche', 0.9784151315689087),
 ('MalWoche', 0.9716570377349854),
 ('woche', 0.9679470062255859),
 ('TageWoche', 0.9576689004898071),
 ('WocheAngela', 0.950394868850708),
 ('StundenWoche', 0.9486403465270996),
 ('Stunden-Woche', 0.947299063205719),
 ('Wocheum', 0.9403384923934937)]